In [1]:
from __future__ import print_function, division, unicode_literals
import pandas as pd
import numpy as np
import os

In [2]:
#Load CSV 
def load_housing_data(housing_path="CaliforniaHousing"):
    csv_path= os.path.join(housing_path,"cal_housing.csv")
    return pd.read_csv(csv_path)

In [3]:
housing= load_housing_data()
housing.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41,880,129.0,322,126,8.3252,452600,NEAR BAY
1,-122.22,37.86,21,7099,1106.0,2401,1138,8.3014,358500,NEAR BAY
2,-122.24,37.85,52,1467,190.0,496,177,7.2574,352100,NEAR BAY
3,-122.25,37.85,52,1274,235.0,558,219,5.6431,341300,NEAR BAY
4,-122.25,37.85,52,1627,280.0,565,259,3.8462,342200,NEAR BAY


In [4]:
#Divide by 1.5 to limit the number of income categories
housing["income_cat"]= np.ceil(housing["median_income"]/1.5)
housing["income_cat"].where(housing["income_cat"]<5,5.0, inplace=True)
housing["income_cat"].value_counts()


from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=123)
for train_index, test_index in split.split(housing, housing["income_cat"]):
    strat_train_set= housing.loc[train_index]
    strat_test_set= housing.loc[test_index]


print(strat_test_set["income_cat"].value_counts()/len(strat_test_set))
print(strat_train_set["income_cat"].value_counts()/len(strat_train_set))
print(housing["income_cat"].value_counts()/len(housing))

#Ratio of split looks good enough to proceed


#Dropping Income_cat
for set_ in (strat_train_set, strat_test_set):
    set_.drop("income_cat", axis=1, inplace=True)
    
strat_train_label= strat_train_set["median_house_value"]
strat_train_set.drop("median_house_value", axis=1, inplace=True)

strat_train_label.head()
strat_train_set.head()

3.0    0.350533
2.0    0.318798
4.0    0.176357
5.0    0.114583
1.0    0.039729
Name: income_cat, dtype: float64
3.0    0.350594
2.0    0.318859
4.0    0.176296
5.0    0.114402
1.0    0.039850
Name: income_cat, dtype: float64
3.0    0.350581
2.0    0.318847
4.0    0.176308
5.0    0.114438
1.0    0.039826
Name: income_cat, dtype: float64


,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity
10888,-117.90,33.70,12,4695,1110.0,2153,989,4.6483,<1H OCEAN
2092,-119.78,36.75,43,2070,512.0,1925,444,1.4635,INLAND
18810,-121.67,40.89,17,2548,537.0,1118,461,2.2670,INLAND
6905,-118.11,34.03,36,1493,316.0,989,293,3.5272,<1H OCEAN
10102,-117.97,33.93,35,1887,328.0,989,351,4.1321,<1H OCEAN


In [5]:
housing= strat_train_set.copy()
housing_num =housing.drop("ocean_proximity", axis=1)
housing_num.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income
10888,-117.90,33.70,12,4695,1110.0,2153,989,4.6483
2092,-119.78,36.75,43,2070,512.0,1925,444,1.4635
18810,-121.67,40.89,17,2548,537.0,1118,461,2.2670
6905,-118.11,34.03,36,1493,316.0,989,293,3.5272
10102,-117.97,33.93,35,1887,328.0,989,351,4.1321


In [6]:
#Custom Transformer

from sklearn.base import BaseEstimator, TransformerMixin

rooms_ix, bedrooms_ix, population_ix, household_ix= 3,4,5,6

class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_bedrooms_per_room=True):
        self.add_bedrooms_per_room= add_bedrooms_per_room
    def fit(self, X, y=None):
        return self
    def transform(self, X, Y=None):
        rooms_per_household= X[:,rooms_ix]/X[:,household_ix]
        population_per_household=X[:,population_ix]/X[:,household_ix]
        if self.add_bedrooms_per_room:
            bedrooms_per_room= X[:, bedrooms_ix]/X[:,rooms_ix]
            return np.c_[X, rooms_per_household, population_per_household, bedrooms_per_room]
        else:
            return np.c_[X, rooms_per_household, population_per_household]

In [7]:
attr_adder= CombinedAttributesAdder(add_bedrooms_per_room=False)
housing_extra_attribs= attr_adder.transform(housing.values)

In [8]:
#Transformation Pipeline

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, Imputer, LabelBinarizer

num_pipeline= Pipeline([
    ('imputer', Imputer(strategy='median')),
    ('attribs_adder', CombinedAttributesAdder()),
    ('std_scaler', StandardScaler())
])

In [9]:
housing_num_tr=num_pipeline.fit_transform(housing_num)
housing_num_tr=pd.DataFrame(
    housing_extra_attribs,
    columns=list(housing.columns)+["rooms_per_household", "population_per_household"])
housing_num_tr.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity,rooms_per_household,population_per_household
0,-117.9,33.7,12,4695,1110,2153,989,4.6483,<1H OCEAN,4.74722,2.17695
1,-119.78,36.75,43,2070,512,1925,444,1.4635,INLAND,4.66216,4.33559
2,-121.67,40.89,17,2548,537,1118,461,2.267,INLAND,5.52711,2.42516
3,-118.11,34.03,36,1493,316,989,293,3.5272,<1H OCEAN,5.09556,3.37543
4,-117.97,33.93,35,1887,328,989,351,4.1321,<1H OCEAN,5.37607,2.81766


In [10]:
from sklearn.base import BaseEstimator, TransformerMixin

class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names= attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.attribute_names].values

In [13]:
num_attribs= list(housing_num)
print(list(housing_num))
cat_attribs= ["ocean_proximity"]

num_pipeline= Pipeline([
    ('selector', DataFrameSelector(num_attribs)),
    ('imputer', Imputer(strategy='median')),
    ('attribs_adder', CombinedAttributesAdder()),
    ('std_scaler', StandardScaler())
])

#cat_pipeline= Pipeline([
#    ('selector', DataFrameSelector(cat_attribs)),
#    ('label_binarizer', LabelBinarizer()),
#])

from sklearn_pandas import DataFrameMapper
cat_pipeline = Pipeline([
    ('label_binarizer', DataFrameMapper([(cat_attribs, LabelBinarizer())])),
])

['longitude', 'latitude', 'housing_median_age', 'total_rooms', 'total_bedrooms', 'population', 'households', 'median_income']


In [67]:
from sklearn.pipeline import FeatureUnion

full_pipeline= FeatureUnion(transformer_list=[
    ("num_pipeline", num_pipeline),
    ("cat_pipeline", cat_pipeline)
])

In [68]:
housing_prepared=full_pipeline.fit_transform(housing)
housing_prepared

array([[ 0.83777132, -0.90910198, -1.31880165, ...,  0.        ,
         0.        ,  0.        ],
       [-0.0982453 ,  0.5143055 ,  1.14388773, ...,  0.        ,
         0.        ,  0.        ],
       [-1.03924072,  2.44640613, -0.92159368, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 1.26097031, -1.11444601, -1.15991846, ...,  0.        ,
         0.        ,  0.        ],
       [-0.96953735,  1.36834998,  0.3494718 , ...,  0.        ,
         0.        ,  0.        ],
       [-1.11392289,  1.09766921,  0.11114703, ...,  0.        ,
         0.        ,  0.        ]])

In [69]:
housing_prepared=pd.DataFrame(housing_prepared,
                              columns=list(housing_num.columns)+["rooms_per_household", "population_per_household", "bedrooms_per_room","OP_Bin1", 
                                           "OP_Bin2", "OP_Bin3", "OP_Bin4", "OP_Bin5"])
housing_prepared.head()
#housing_prepared.shape

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,rooms_per_household,population_per_household,bedrooms_per_room,OP_Bin1,OP_Bin2,OP_Bin3,OP_Bin4,OP_Bin5
0,0.837771,-0.909102,-1.318802,0.933940,1.355583,0.631030,1.268895,0.404542,-0.265170,-0.078732,0.341397,1.0,0.0,0.0,0.0,0.0
1,-0.098245,0.514305,1.143888,-0.259217,-0.061046,0.432749,-0.146022,-1.260375,-0.297797,0.110060,0.506322,0.0,1.0,0.0,0.0,0.0
2,-1.039241,2.446406,-0.921594,-0.041949,-0.001822,-0.269061,-0.101887,-0.840329,0.033982,-0.057024,-0.046225,0.0,1.0,0.0,0.0,0.0
3,0.733216,-0.755094,0.587797,-0.521484,-0.525359,-0.381247,-0.538044,-0.181535,-0.131553,0.026086,-0.032621,1.0,0.0,0.0,0.0,0.0
4,0.802920,-0.801763,0.508355,-0.342397,-0.496932,-0.381247,-0.387466,0.134689,-0.023956,-0.022696,-0.603955,1.0,0.0,0.0,0.0,0.0


In [70]:
#Implement Linear Regression

from sklearn.linear_model import LinearRegression

housing_labels= strat_train_label.copy()

lin_reg= LinearRegression()
lin_reg.fit(housing_prepared, housing_labels)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [18]:
#Trying out on few instances
some_data= housing_prepared.iloc[:5]
some_labels= housing_labels.iloc[:5]
#some_prepared_data= full_pipeline.fit_transform(some_data)
print("Predictions:", lin_reg.predict(some_data))

Predictions: [274369.41779856  55315.37709485  32410.55033648 204224.47020141
 227822.7362163 ]


In [19]:
print(list(some_labels))

[190800, 46600, 57800, 213700, 198100]


In [20]:
#Check quality of model

from sklearn.metrics import mean_squared_error

housing_predictions= lin_reg.predict(housing_prepared)
lin_mse= mean_squared_error(housing_labels, housing_predictions)
lin_rmse= np.sqrt(lin_mse)
lin_rmse

#Underfitting model

68163.84275048292

In [21]:
#Using Decision Tree Regressor

from sklearn.tree import DecisionTreeRegressor

tree_reg= DecisionTreeRegressor()
tree_reg.fit(housing_prepared, housing_labels)

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

In [22]:
housing_predictions= tree_reg.predict(housing_prepared)
tree_rmse= mean_squared_error(housing_labels, housing_predictions)
tree_rmse= np.sqrt(tree_rmse)
tree_rmse

# Now it is overfitting

0.0

In [23]:
#Using cross validation

from sklearn.model_selection import cross_val_score
scores= cross_val_score(tree_reg, housing_prepared, housing_labels, scoring="neg_mean_squared_error", cv=10)
tree_rmse_scores= np.sqrt(-scores)
tree_rmse_scores

array([71458.86489324, 72249.00587252, 71314.88606173, 70070.98870569,
       71854.76010091, 69828.25495363, 68552.03491696, 70926.34992066,
       70458.03699553, 66999.21172712])

In [24]:
#Check RMSE
def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard Deviation:", scores.std())

In [25]:
#RMSE of Decision Tree

display_scores(tree_rmse_scores)

Scores: [71458.86489324 72249.00587252 71314.88606173 70070.98870569
 71854.76010091 69828.25495363 68552.03491696 70926.34992066
 70458.03699553 66999.21172712]
Mean: 70371.23941479766
Standard Deviation: 1522.530082134769


In [26]:
#RMSE of Linear Regression

lin_scores= cross_val_score(lin_reg, housing_prepared, housing_labels, scoring= "neg_mean_squared_error", cv=10)

lin_rmse_scores= np.sqrt(-lin_scores)
display_scores(lin_rmse_scores)

Scores: [69632.10182642 69118.84646769 66254.64334091 70558.40593379
 68047.48394193 67849.54843674 69175.15807561 69398.05276184
 67851.48398075 67127.03528939]
Mean: 68501.27600550668
Standard Deviation: 1230.4093246429848


In [27]:
#Random Forest

from sklearn.ensemble import RandomForestRegressor

forest_reg= RandomForestRegressor()
forest_reg.fit(housing_prepared, housing_labels)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [30]:
forest_scores= cross_val_score(forest_reg, housing_prepared, housing_labels, scoring= "neg_mean_squared_error", cv=10)

forest_rmse_scores= np.sqrt(-forest_scores)
display_scores(forest_rmse_scores)

Scores: [53047.081584   52437.17585415 52630.61506313 51425.97907198
 51785.88329606 52168.31086419 54878.27218547 52486.02316572
 53549.0994751  51841.83630372]
Mean: 52625.027686352296
Standard Deviation: 955.3873877819001


In [37]:
#Using SVM

from sklearn.externals import svm

svm_mod= svm.SVR(kernel="linear")

svm_mod.fit(housing_prepared, housing_labels)

SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
  kernel='linear', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [38]:
svm_scores= cross_val_score(svm_mod, housing_prepared, housing_labels, scoring="neg_mean_squared_error", cv=10)

svm_rmse_scores= np.sqrt(-svm_scores)
display_scores(svm_rmse_scores)

Scores: [112194.10978372 114639.91172456 109941.91788847 114193.88437148
 110061.45265057 113092.81760769 107317.40504034 112636.83615833
 107855.68890267 111298.58109033]
Mean: 111323.26052181679
Standard Deviation: 2375.450586273491


In [40]:
#Save models

from sklearn.externals import joblib

joblib.dump(forest_reg, "Random_Forest_model.pkl")
forest_reg_model_loaded= joblib.load("Random_Forest_model.pkl")

In [42]:
#Fine Tune Model

#Grid Search

from sklearn.model_selection import GridSearchCV

param_grid=[
    {'n_estimators':[3,10,30], 'max_features':[2,4,6,8]},
    {'bootstrap': [False], 'n_estimators':[3,10], 'max_features':[2,3,4]}
]

forest_reg= RandomForestRegressor()

grid_search= GridSearchCV(forest_reg, param_grid, cv=5,
                         scoring= 'neg_mean_squared_error')

grid_search.fit(housing_prepared, housing_labels)

GridSearchCV(cv=5, error_score='raise',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid=[{'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6, 8]}, {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_squared_error', verbose=0)

In [43]:
grid_search.best_params_

{'max_features': 6, 'n_estimators': 30}

In [45]:
grid_search.best_estimator_

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features=6, max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=30, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)

In [48]:
cvres= grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)

63285.11714903132 {'max_features': 2, 'n_estimators': 3}
54944.589592020384 {'max_features': 2, 'n_estimators': 10}
52328.40728231476 {'max_features': 2, 'n_estimators': 30}
60040.59877990803 {'max_features': 4, 'n_estimators': 3}
52345.7944551594 {'max_features': 4, 'n_estimators': 10}
49988.27142454352 {'max_features': 4, 'n_estimators': 30}
59262.22657449357 {'max_features': 6, 'n_estimators': 3}
51415.32570026715 {'max_features': 6, 'n_estimators': 10}
49801.04936345951 {'max_features': 6, 'n_estimators': 30}
58703.79468730698 {'max_features': 8, 'n_estimators': 3}
52488.316794285405 {'max_features': 8, 'n_estimators': 10}
49867.1356008825 {'max_features': 8, 'n_estimators': 30}
61184.06073724655 {'bootstrap': False, 'max_features': 2, 'n_estimators': 3}
54051.670250615454 {'bootstrap': False, 'max_features': 2, 'n_estimators': 10}
59619.6327219761 {'bootstrap': False, 'max_features': 3, 'n_estimators': 3}
52505.99725641878 {'bootstrap': False, 'max_features': 3, 'n_estimators': 10

In [52]:
grid_search.fit(housing_prepared, housing_labels)

GridSearchCV(cv=5, error_score='raise',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid=[{'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6, 8]}, {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_squared_error', verbose=0)

In [54]:
grid_search.best_params_

{'max_features': 6, 'n_estimators': 30}

In [55]:
#Using Fine tuned model to predict

grid_search.predict(housing_prepared.iloc[:5])

array([189546.66666667,  47626.66666667,  74393.33333333, 203903.33333333,
       206366.66666667])

In [59]:
from sklearn.model_selection import RandomizedSearchCV

param_grid={
    'n_estimators':[3,10,30], 'max_features':[2,4,6,8]
}


forest_reg= RandomForestRegressor()

random_search= RandomizedSearchCV(forest_reg, param_grid, cv=5,
                         scoring= 'neg_mean_squared_error')

random_search.fit(housing_prepared, housing_labels)

RandomizedSearchCV(cv=5, error_score='raise',
          estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False),
          fit_params=None, iid=True, n_iter=10, n_jobs=1,
          param_distributions={'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6, 8]},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score='warn', scoring='neg_mean_squared_error',
          verbose=0)

In [60]:
cvres_random= random_search.cv_results_
for mean_score, params in zip(cvres_random["mean_test_score"], cvres_random["params"]):
    print(np.sqrt(-mean_score), params)

52868.404570193925 {'n_estimators': 10, 'max_features': 4}
63315.83363486005 {'n_estimators': 3, 'max_features': 2}
50290.83030479053 {'n_estimators': 30, 'max_features': 4}
49890.00323779991 {'n_estimators': 30, 'max_features': 6}
52555.26903505521 {'n_estimators': 30, 'max_features': 2}
49950.56792252016 {'n_estimators': 30, 'max_features': 8}
51967.62625090134 {'n_estimators': 10, 'max_features': 8}
60765.22740125981 {'n_estimators': 3, 'max_features': 4}
52417.71708706354 {'n_estimators': 10, 'max_features': 6}
58936.49802010342 {'n_estimators': 3, 'max_features': 8}


In [61]:
random_search.best_params_

{'n_estimators': 30, 'max_features': 6}

In [63]:
#Ensemble Method

#Analyze best models and their errors

feature_importances= grid_search.best_estimator_.feature_importances_
feature_importances

array([7.70655645e-02, 6.73798927e-02, 4.16326955e-02, 1.68430398e-02,
       1.68192447e-02, 1.71631045e-02, 1.62377428e-02, 3.15261224e-01,
       7.08193413e-02, 1.08840625e-01, 7.87917117e-02, 9.38054187e-03,
       1.56422541e-01, 1.70182216e-04, 2.73886062e-03, 4.43368710e-03])

In [74]:
#extra_attribs= ["rooms_per_household", "population_per_household", "bedrooms_per_room"]
#cat_onehot_attribs= list(encoder.classes_)
#attributes= num_attribs+extra_attribs+cat_onehot_attribs
attributes=list(housing_prepared)
sorted(zip(feature_importances, attributes), reverse=True)

[(0.315261224023521, 'median_income'),
 (0.1564225413637138, 'OP_Bin2'),
 (0.10884062546333624, 'population_per_household'),
 (0.0787917116811049, 'bedrooms_per_room'),
 (0.0770655644681697, 'longitude'),
 (0.07081934126142872, 'rooms_per_household'),
 (0.06737989271929679, 'latitude'),
 (0.04163269548330024, 'housing_median_age'),
 (0.01716310446571208, 'population'),
 (0.01684303976701824, 'total_rooms'),
 (0.016819244707853627, 'total_bedrooms'),
 (0.016237742793853985, 'households'),
 (0.00938054187447445, 'OP_Bin1'),
 (0.004433687095790635, 'OP_Bin5'),
 (0.002738860615675681, 'OP_Bin4'),
 (0.0001701822157499239, 'OP_Bin3')]

In [77]:
#Evaluate model on test data
final_model= grid_search.best_estimator_

X_test= strat_test_set.drop("median_house_value", axis=1)
y_test= strat_test_set["median_house_value"].copy()

In [79]:
X_test_prepared= full_pipeline.transform(X_test)

In [80]:
final_prediction= final_model.predict(X_test_prepared)

In [81]:
final_mse= mean_squared_error(y_test, final_prediction)
final_rmse= np.sqrt(final_mse)

In [82]:
final_rmse

49738.173173567004